# Game Summary

In [13]:
import requests
from datetime import timedelta, date, datetime
import json
import pandas as pd
import os
import csv
import sys
import time

In [14]:
sys.path.append('../../code')  # Replace '/path/to/code' with the actual path to your 'code' directory
from scrape_new import daterange, get_daily_game_ids

In [15]:
def get_game_summary(api_key, game_id, delay_seconds=1):
    '''
    Fetches and extracts game summary information from the Sportradar API.

    Params:
    api_key: str - API key for accessing the Sportradar service
    game_id: str - Unique identifier for the game
    delay_seconds: int - Number of seconds to wait after the request is made

    Returns:
    game: dict - Dictionary containing formatted game summary if successful, None otherwise
    '''
    url = f'https://api.sportradar.com/mlb/trial/v7/en/games/{game_id}/summary.json'
    response = requests.get(url, params={'api_key': api_key})

    # Delay after making the request
    time.sleep(delay_seconds)

    # Check if the request was successful
    if response.status_code == 200:
        game_summary = response.json()
        game_data = game_summary.get('game', {})
        
        # Extracting team abbreviations and date information
        away_abbr = game_data.get('away', {}).get('abbr', 'UNK')
        home_abbr = game_data.get('home', {}).get('abbr', 'UNK')
        scheduled_time = game_data.get('scheduled', 'No scheduled time found')
        date = scheduled_time.split('T')[0]
        day_night = game_data.get('day_night', 'Unknown')

        # Constructing game_id
        custom_game_id = f"{away_abbr}{home_abbr}{date.replace('-', '')}{day_night}"

        # Forming the game dictionary
        game = {
            'away_team_abbr': away_abbr,
            'home_team_abbr': home_abbr,
            'date': date,
            'start_time': scheduled_time.split('T')[1][:5] if 'T' in scheduled_time else 'Unknown',
            'day_night': day_night,
            'venue': game_data.get('venue', {}).get('name', 'No venue name found'),
            'field_type': game_data.get('venue', {}).get('stadium_type', 'No stadium type found'),
            'away_team_errors': game_data.get('away', {}).get('errors', 0),
            'home_team_errors': game_data.get('home', {}).get('errors', 0),
            'away_team_hits': game_data.get('away', {}).get('hits', 0),
            'home_team_hits': game_data.get('home', {}).get('hits', 0),
            'away_team_runs': game_data.get('away', {}).get('runs', 0),
            'home_team_runs': game_data.get('home', {}).get('runs', 0),
            'game_id': custom_game_id
        }
        
        return game
    else:
        print(f"Failed to retrieve data for game ID {game_id}: Status Code {response.status_code}")
        return None


In [16]:
# Define the function to save game summaries to a CSV file
def save_game_summaries_to_csv(game_summaries, filename):
    # Define the CSV fieldnames based on the keys in the game summary dictionary
    fieldnames = game_summaries[0].keys() if game_summaries else []
    
    # Write the game summaries to the CSV file
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(game_summaries)

In [20]:
# Example usage
api_key = 'iA8kUIXIOH8ko6SF2q1z42hpKLJWA4Hj4fajST1i'
start_date = datetime(2024, 4, 27).date()
filename = '../../data/game_summaries1.csv'

# Ensure the directory exists, create if not
os.makedirs(os.path.dirname(filename), exist_ok=True)

header_written = not os.path.exists(filename)  # Determine if the header needs to be written

for single_date in daterange(start_date):
    year = single_date.year
    month = f"{single_date.month:02d}"
    day = f"{single_date.day:02d}"
    game_ids = get_daily_game_ids(api_key, year, month, day)
    for game_id in game_ids:
        game_summary = get_game_summary(api_key, game_id, delay_seconds=1)
        if game_summary:
            # Save each game summary immediately to the CSV file
            df = pd.DataFrame([game_summary])
            df.to_csv(filename, mode='a', index=False, header=header_written)
            header_written = False  # Update header_written to avoid writing the header again

print(f"Game summaries saved to {filename}")


Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/27/summary.json | Status Code: 200
Games found on 2024-04-27: 15
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/28/summary.json | Status Code: 200
Games found on 2024-04-28: 15
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/29/summary.json | Status Code: 200
Games found on 2024-04-29: 12
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/30/summary.json | Status Code: 200
Games found on 2024-04-30: 16
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/05/01/summary.json | Status Code: 200
Games found on 2024-05-01: 15
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/05/02/summary.json | Status Code: 200
Games found on 2024-05-02: 6
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/05/03/summary.json | Status Code: 200
Games found on 2024-05-03: 15
Request URL: https://api.sportradar.com/mlb/trial/v7/en/